## read in data

<i>format as test and train

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import keras

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train_small.csv', dtype={"from_2": object, "type": object, "category": object})

In [3]:
train.dropna(subset=['type'], inplace=True)

In [4]:
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
test.drop(columns=['text_type'], axis=1, inplace=True)

In [7]:
test['category']=np.nan

In [8]:
X = train['text']
y = train['category']

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 2, test_size=0.2)

## prepare data

<i> label encode and one-hot encode categories

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [11]:
num_classes=6
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

<i> use sklearn's vectorizer to format text data at word level

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)
print (x_train_vec.shape)
print (x_test_vec.shape)

(207, 2277)
(52, 2277)


## hyperparameter training

In [13]:
# Utility function to report best scores- from kaggle:https://www.kaggle.com/ksjpswaroop/parameter-tuning-rf-randomized-search
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import RandomizedSearchCV as RS
from scipy import stats
from scipy.stats import randint
from scipy.stats import uniform

In [22]:
from keras.constraints import maxnorm
from keras.layers import Dropout

def create_model_2(dropout_rate=0.0, weight_constraint=0, hidden_layers=1, neurons=1):
    model2 = Sequential()
    model2.add(Dense(neurons, activation='relu', input_shape=(2277,)))
    for i in np.arange(hidden_layers):
        model2.add(Dense(neurons, activation='relu'))
        model2.add(Dropout(dropout_rate))
    model2.add(Dense(6, activation='softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2

In [23]:
model2 = KerasClassifier(build_fn=create_model_2, verbose=0, shuffle=True)

In [24]:
params = {'neurons': stats.randint(2, 52),
         'hidden_layers': stats.randint(1, 5),
         'dropout_rate': stats.uniform(0.0, 0.9),
         'weight_constraint': stats.uniform(1, 5)
         }
n_iter = 64

In [25]:
rand2 = RS(estimator=model2, param_distributions=params, n_jobs=-1, cv=4, n_iter=n_iter)

In [26]:
rand_search2 = rand2.fit(x_train_vec, y_train)

In [27]:
report(rand_search2.cv_results_)

Model with rank: 1
Mean validation score: 0.585 (std: 0.085)
Parameters: {'dropout_rate': 0.04001235283104452, 'hidden_layers': 1, 'neurons': 48, 'weight_constraint': 4.905616934045857}

Model with rank: 2
Mean validation score: 0.580 (std: 0.060)
Parameters: {'dropout_rate': 0.37003997759641916, 'hidden_layers': 3, 'neurons': 34, 'weight_constraint': 3.4725706521407225}

Model with rank: 3
Mean validation score: 0.570 (std: 0.058)
Parameters: {'dropout_rate': 0.6957291048549885, 'hidden_layers': 2, 'neurons': 30, 'weight_constraint': 5.473048038692594}

Model with rank: 4
Mean validation score: 0.570 (std: 0.070)
Parameters: {'dropout_rate': 0.5401694315326324, 'hidden_layers': 4, 'neurons': 23, 'weight_constraint': 1.7849140259991705}

Model with rank: 4
Mean validation score: 0.570 (std: 0.058)
Parameters: {'dropout_rate': 0.4671889522366334, 'hidden_layers': 3, 'neurons': 39, 'weight_constraint': 1.9535786553641223}



from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

from sklearn.model_selection import GridSearchCV

batch_size= [4,8,16,32,64,128,256]
epochs=[4,8,16,32,64,128]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

grid_pred = grid.fit(x_train_vec, y_train);

print(grid_pred.best_score_)
print(grid_pred.best_params_)

model.fit(x=x_train_vec, y=y_train, batch_size=?, epochs=?)
#test_loss, test_acc = model.evaluate(x_test_vec, y_test)
#test_acc

def create_model_2(optimizer='Adam'):
    inputs = layers.Input(shape=(2163,))
    hidden_1 = layers.Dense(units=32, activation='relu')(inputs)
    dropout1 = layers.Dropout(0.5)(hidden_1)
    hidden_2 = layers.Dense(units=32, activation='relu')(hidden_1)
    dropout_2 = layers.Dropout(0.5)(hidden_2)
    hidden_3 = layers.Dense(units=32, activation='relu')(hidden_2)
    dropout_3 = layers.Dropout(0.5)(hidden_3)
    hidden_4 = layers.Dense(units=32, activation='relu')(hidden_3)
    dropout_4 = layers.Dropout(0.5)(hidden_3)
    hidden_5 = layers.Dense(units=32, activation='relu')(hidden_4)
    dropout_5 = layers.Dropout(0.5)(hidden_4)
    outputs = layers.Dense(6, activation='softmax')(hidden_5)

    model2= models.Model(inputs=inputs, outputs=outputs)
    #model.summary()
    model2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
    return model2

from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model_2, verbose=0, batch_size=32, epochs=64)

from sklearn.model_selection import GridSearchCV

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

grid_pred = grid.fit(x_train_vec, y_train);

print(grid_pred.best_score_)
print(grid_pred.best_params_)